In [ ]:
import yfinance as yf
import pandas as pd
import os
import sys

# Add project root to path if not already present
project_root = os.path.dirname(os.path.abspath(''))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import modular utilities
from config import Config
from index_utils import create_index_processor

# Initialize secure configuration
config = Config()

# Get configuration values
index_symbols = config.get_index_symbols()
start_date = config.get('START_DATE')
end_date = config.get('END_DATE')
data_save_path = config.get_data_save_path()

print(f"Fetching data from {start_date} to {end_date} for {len(index_symbols)} indices")
print(f"Data will be saved to: {data_save_path}")

# Fetch index data using configuration
for name, symbol in index_symbols.items():
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval="1d")
        file_path = os.path.join(data_save_path, f"{name}_history.csv")
        df.to_csv(file_path)
        print(f"Fetched {len(df)} rows for {name}")
    except Exception as e:
        print(f"Failed to fetch {name}: {e}")

print("All index data saved successfully.")

# Process the data using modular utilities
processor = create_index_processor(data_save_path)

# Execute complete data processing pipeline
final_enriched_data = processor.process_complete_pipeline()

print(f"\nData processing completed successfully!")
print(f"Final dataset shape: {final_enriched_data.shape}")
print(f"Total columns: {len(final_enriched_data.columns)}")
print(f"Date range: {final_enriched_data.index.min()} to {final_enriched_data.index.max()}")

# Show sample of the processed data
print(f"\nFirst 5 rows of processed data:")
print(final_enriched_data.head())

# Save the final processed data
final_data_path = os.path.join(data_save_path, "processed_index_data.csv")
final_enriched_data.to_csv(final_data_path)
print(f"\nProcessed data saved to: {final_data_path}")